# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# string for output path
out_path = "../output_data/"

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(f"{out_path}cities.csv")
# Temp conversion from K to F
city_data_df['Max Temp'] = (city_data_df['Max Temp'] - 273.15) * 1.8 + 32

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,64.778,88,100,2.24,PT,1699832766
1,1,rio grande,-32.0350,-52.0986,62.708,95,100,2.75,BR,1699832767
2,2,kavaratti,10.5669,72.6420,82.040,75,70,1.68,IN,1699832767
3,3,talnakh,69.4865,88.3972,29.210,96,100,4.29,RU,1699832768
4,4,hithadhoo,-0.6000,73.0833,81.302,79,93,2.52,MV,1699832769


In [6]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     573 non-null    int64  
 1   City        573 non-null    object 
 2   Lat         573 non-null    float64
 3   Lng         573 non-null    float64
 4   Max Temp    573 non-null    float64
 5   Humidity    573 non-null    int64  
 6   Cloudiness  573 non-null    int64  
 7   Wind Speed  573 non-null    float64
 8   Country     571 non-null    object 
 9   Date        573 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 44.9+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values

# Between 68 and 80 degrees F, wind speed of less than 3 m/s, and cloud cover less than 30% but greater than 5%
# Dropping any rows with null values in the same line (technically...?)
filtered_df = city_data_df.loc[(city_data_df['Max Temp'] >= 68.0) 
                               & (city_data_df['Max Temp'] <= 80.0) 
                               & (city_data_df['Wind Speed'] < 3.0) 
                               & (city_data_df['Cloudiness'] < 30)
                               & (city_data_df['Cloudiness'] > 5),:].dropna()

# The above filters are rather arbitrary but did get the filtered data to less than 10 rows

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,kantharalak,14.6406,104.6499,77.972,100,16,1.05,TH,1699832799
111,111,boundiali,9.5217,-6.4869,78.296,57,21,1.34,CI,1699832851
344,344,antonio enes,-16.2325,39.9086,76.136,83,13,1.39,MZ,1699833031
362,362,howrah,22.5892,88.3103,69.800,88,28,2.06,IN,1699833044
443,443,sensuntepeque,13.8667,-88.6333,71.402,87,18,0.97,SV,1699833102
498,498,beau vallon,-20.4189,57.6953,71.222,94,20,2.57,MU,1699833144
543,543,beira,-19.8436,34.8389,69.890,100,20,1.54,MZ,1699833177


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
41,kantharalak,TH,14.6406,104.6499,100,
111,boundiali,CI,9.5217,-6.4869,57,
344,antonio enes,MZ,-16.2325,39.9086,83,
362,howrah,IN,22.5892,88.3103,88,
443,sensuntepeque,SV,13.8667,-88.6333,87,
498,beau vallon,MU,-20.4189,57.6953,94,
543,beira,MZ,-19.8436,34.8389,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel

# 10km radius
radius = 10000

params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "type": 'amenity',
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Set base URL (This does NOT need to be inside the loop)
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city_lat = row['Lat']
    city_lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{city_lng},{city_lat},{radius}"
    params["bias"] = f"proximity:{city_lng},{city_lat}"

    # Make and API request using the params dictionary
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kantharalak - nearest hotel: SB Hôtel
boundiali - nearest hotel: No hotel found
antonio enes - nearest hotel: Hotel Quirimbas
howrah - nearest hotel: Sun India Guest House
sensuntepeque - nearest hotel: Hotel Arizona
beau vallon - nearest hotel: Le Jardin de Beau Vallon
beira - nearest hotel: Hotel Savoy


,City,Country,Lat,Lng,Humidity,Hotel Name
41,kantharalak,TH,14.6406,104.6499,100,SB Hôtel
111,boundiali,CI,9.5217,-6.4869,57,No hotel found
344,antonio enes,MZ,-16.2325,39.9086,83,Hotel Quirimbas
362,howrah,IN,22.5892,88.3103,88,Sun India Guest House
443,sensuntepeque,SV,13.8667,-88.6333,87,Hotel Arizona
498,beau vallon,MU,-20.4189,57.6953,94,Le Jardin de Beau Vallon
543,beira,MZ,-19.8436,34.8389,100,Hotel Savoy


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)